In [115]:
import pandas as pd
import datetime
import os
import matplotlib.pyplot as plt
import requests
import numpy as np

In [93]:
def get_osm_map_image(latitude, longitude, zoom=15, width=600, height=400):
    base_url = "https://www.openstreetmap.org/export/embed"
    
    # Set parameters for the API request
    params = {
        'bbox': f'{longitude-0.005},{latitude-0.005},{longitude+0.005},{latitude+0.005}',
        'layer': 'mapnik',
        'marker': f'{latitude},{longitude}',
        'zoom': zoom,
        'width': width,
        'height': height
    }

    # Make the API request
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        # Save the map image to a file
        with open("osm_map_image.html", "w") as file:
            file.write(response.text)
        
        
    else:
        print(f"Failed to retrieve map image. Status code: {response.status_code}")

In [116]:
df = pd.read_excel('HEDNO_KPIs_v3.xlsx', engine='openpyxl')
df.head()

,Installation,Transformer,Distribution,Total energy consumption (MWh),Nr. of power alarms L1,Min duration of power alarms L1 (min),Max duration of power alarms L1 (min),Nr. of power alarms L2,Min duration of power alarms L2 (min),Max duration of power alarms L2 (min),...,% of time of Current unbalance alarms,Nr. of Power Fails (outage),Min time of Power Fails (sec),Max time of Power Fails (sec),Nr. of Voltage dips,Avg time of Voltage dips (msec),Nr. of Voltage swells,Avg time of Voltage swells (msec),Latitude,Longitude
0,Ρ 3-19 ΜΑΡΚΟΠΟΥΛΟΥ,ΜΜ-423,NaN,13.98,9.0,1.0,10.0,89.0,1.0,129.0,...,0.00,0,0.000,0.000,6,328.000000,0,0.0,37.889782,23.915392
1,Ρ 3-19 ΜΑΡΚΟΠΟΥΛΟΥ,ΜΜ-423,102.408.000766,0.91,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,0.000,0.000,6,216.000000,0,0.0,37.889782,23.915392
2,Ρ 3-19 ΜΑΡΚΟΠΟΥΛΟΥ,ΜΜ-423,102.408.000767,0.02,NaN,NaN,NaN,NaN,NaN,NaN,...,0.99,0,0.000,0.000,7,281.285714,0,0.0,37.889782,23.915392
3,ΜΠ-161 ΠΑΛΛΗΝΗΣ,ΜΠ-297,NaN,109.98,76.0,1.0,39.0,26.0,1.0,8.0,...,0.00,0,0.000,0.000,2,225.000000,0,0.0,37.969293,23.853301
4,ΜΠ-22 Κ/Δ ΠΑΛΛΗΝΗΣ,ΜΠ-013,NaN,200.57,701.0,1.0,625.0,665.0,1.0,664.0,...,0.04,1,537.999,537.999,5,67.400000,0,0.0,37.958051,23.852520


In [120]:
###### TEST to get new columns

grouped = df.groupby('Transformer')
for i, group in grouped:
    
    group['Nr. of feeder overcurrent alarms'] = group['Nr. of overcurrent alarms L1']+group['Nr. of overcurrent alarms L2']+group['Nr. of overcurrent alarms L3']
    group['Min duration of feeder overcurrent alarms (minutes)'] = group[['Min duration of overcurrent alarms L1 (min)','Min duration of overcurrent alarms L2 (min)','Min duration of overcurrent alarms L3 (min)']].min(axis=1)
    group['Max duration of feeder overcurrent alarms (minutes)'] = group[['Max duration of overcurrent alarms L1 (min)','Max duration of overcurrent alarms L2 (min)','Max duration of overcurrent alarms L3 (min)']].max(axis=1)
    #group['Min duration of feeder overcurrent alarms'] = np.min(group['Min duration of overcurrent alarms L1 (min)'],group['Min duration of overcurrent alarms L2 (min)'],group['Min duration of overcurrent alarms L3 (min)'])
    #group['Max duration of feeder overcurrent alarms'] = np.max(group['Max duration of overcurrent alarms L1 (min)'],group['Max duration of overcurrent alarms L2 (min)'],group['Max duration of overcurrent alarms L3 (min)'])
    
    group['Nr. of feeder overcurrent alarms'] = group['Nr. of feeder overcurrent alarms'].sum()
    group['Min duration of feeder overcurrent alarms (minutes)'] = group['Min duration of feeder overcurrent alarms (minutes)'].min()
    group['Max duration of feeder overcurrent alarms (minutes)'] = group['Max duration of feeder overcurrent alarms (minutes)'].max()

    group['Nr. of power alarms'] = group['Nr. of power alarms L1']+group['Nr. of power alarms L2']+group['Nr. of power alarms L3']
    group['Min duration of power alarms (minutes)'] = group[['Min duration of power alarms L1 (min)','Min duration of power alarms L2 (min)','Min duration of power alarms L3 (min)']].min(axis=1)
    group['Max duration of power alarms (minutes)'] = group[['Max duration of power alarms L1 (min)','Max duration of power alarms L2 (min)','Max duration of power alarms L3 (min)']].max(axis=1)
    
    #group['Min duration of power alarms'] = np.min(group['Min duration of power alarms L1 (min)'],group['Min duration of power alarms L2 (min)'],group['Min duration of power alarms L3 (min)'])
    #group['Max duration of power alarms'] = np.max(group['Max duration of power alarms L1 (min)'],group['Max duration of power alarms L2 (min)'],group['Max duration of power alarms L3 (min)'])

    group = group.drop(['Nr. of overcurrent alarms L1','Nr. of overcurrent alarms L2','Nr. of overcurrent alarms L3','Min duration of overcurrent alarms L1 (min)','Max duration of overcurrent alarms L1 (min)','Min duration of overcurrent alarms L2 (min)','Max duration of overcurrent alarms L2 (min)','Min duration of overcurrent alarms L3 (min)','Max duration of overcurrent alarms L3 (min)'],axis=1)
    group = group.drop(['Nr. of power alarms L1','Nr. of power alarms L2','Nr. of power alarms L3','Min duration of power alarms L1 (min)','Max duration of power alarms L1 (min)','Min duration of power alarms L2 (min)','Max duration of power alarms L2 (min)','Min duration of power alarms L3 (min)','Max duration of power alarms L3 (min)'],axis=1)
    group = pd.DataFrame(group.iloc[0]).transpose()
    columns = group.columns 
    # print(columns)

In [121]:
columns



Index(['Installation', 'Transformer', 'Distribution',
       'Total energy consumption (MWh)', 'Nr. of Voltage unbalance alarms',
       '% of time of Voltage unbalance alarms',
       'Nr. of Current unbalance alarms',
       '% of time of Current unbalance alarms', 'Nr. of Power Fails (outage)',
       'Min time of Power Fails (sec)', 'Max time of Power Fails (sec)',
       'Nr. of Voltage dips', 'Avg time of Voltage dips (msec)',
       'Nr. of Voltage swells', 'Avg time of Voltage swells (msec)',
       'Latitude', 'Longitude', 'Nr. of feeder overcurrent alarms',
       'Min duration of feeder overcurrent alarms (minutes)',
       'Max duration of feeder overcurrent alarms (minutes)',
       'Nr. of power alarms', 'Min duration of power alarms (minutes)',
       'Max duration of power alarms (minutes)'],
      dtype='object')

In [123]:
summary = pd.DataFrame(columns=columns)


grouped = df.groupby('Transformer')
for i, group in grouped:
    group['Nr. of feeder overcurrent alarms'] = group['Nr. of overcurrent alarms L1']+group['Nr. of overcurrent alarms L2']+group['Nr. of overcurrent alarms L3']
    group['Min duration of feeder overcurrent alarms (minutes)'] = group[['Min duration of overcurrent alarms L1 (min)','Min duration of overcurrent alarms L2 (min)','Min duration of overcurrent alarms L3 (min)']].min(axis=1)
    group['Max duration of feeder overcurrent alarms (minutes)'] = group[['Max duration of overcurrent alarms L1 (min)','Max duration of overcurrent alarms L2 (min)','Max duration of overcurrent alarms L3 (min)']].max(axis=1)
    #group['Min duration of feeder overcurrent alarms'] = np.min(group['Min duration of overcurrent alarms L1 (min)'],group['Min duration of overcurrent alarms L2 (min)'],group['Min duration of overcurrent alarms L3 (min)'])
    #group['Max duration of feeder overcurrent alarms'] = np.max(group['Max duration of overcurrent alarms L1 (min)'],group['Max duration of overcurrent alarms L2 (min)'],group['Max duration of overcurrent alarms L3 (min)'])
    
    group['Nr. of feeder overcurrent alarms'] = group['Nr. of feeder overcurrent alarms'].sum()
    group['Min duration of feeder overcurrent alarms (minutes)'] = group['Min duration of feeder overcurrent alarms (minutes)'].min()
    group['Max duration of feeder overcurrent alarms (minutes)'] = group['Max duration of feeder overcurrent alarms (minutes)'].max()

    group['Nr. of power alarms'] = group['Nr. of power alarms L1']+group['Nr. of power alarms L2']+group['Nr. of power alarms L3']
    group['Min duration of power alarms (minutes)'] = group[['Min duration of power alarms L1 (min)','Min duration of power alarms L2 (min)','Min duration of power alarms L3 (min)']].min(axis=1)
    group['Max duration of power alarms (minutes)'] = group[['Max duration of power alarms L1 (min)','Max duration of power alarms L2 (min)','Max duration of power alarms L3 (min)']].max(axis=1)
    
    #group['Min duration of power alarms'] = np.min(group['Min duration of power alarms L1 (min)'],group['Min duration of power alarms L2 (min)'],group['Min duration of power alarms L3 (min)'])
    #group['Max duration of power alarms'] = np.max(group['Max duration of power alarms L1 (min)'],group['Max duration of power alarms L2 (min)'],group['Max duration of power alarms L3 (min)'])

    group = group.drop(['Nr. of overcurrent alarms L1','Nr. of overcurrent alarms L2','Nr. of overcurrent alarms L3','Min duration of overcurrent alarms L1 (min)','Max duration of overcurrent alarms L1 (min)','Min duration of overcurrent alarms L2 (min)','Max duration of overcurrent alarms L2 (min)','Min duration of overcurrent alarms L3 (min)','Max duration of overcurrent alarms L3 (min)'],axis=1)
    group = group.drop(['Nr. of power alarms L1','Nr. of power alarms L2','Nr. of power alarms L3','Min duration of power alarms L1 (min)','Max duration of power alarms L1 (min)','Min duration of power alarms L2 (min)','Max duration of power alarms L2 (min)','Min duration of power alarms L3 (min)','Max duration of power alarms L3 (min)'],axis=1)
    group = pd.DataFrame(group.iloc[0]).transpose()

    summary = summary.append(group, ignore_index=True)

C:\Users\emily\AppData\Local\Temp\ipykernel_19444\673294417.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary = summary.append(group, ignore_index=True)
C:\Users\emily\AppData\Local\Temp\ipykernel_19444\673294417.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary = summary.append(group, ignore_index=True)
C:\Users\emily\AppData\Local\Temp\ipykernel_19444\673294417.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary = summary.append(group, ignore_index=True)
C:\Users\emily\AppData\Local\Temp\ipykernel_19444\673294417.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary = summary.append(group, ignore_

In [124]:
# summary = pd.DataFrame(columns=columns)


# grouped = df.groupby('Transformer')
# for i, group in grouped:
    
#     group['Nr. of feeder overcurrent alarms'] = group['Nr. of overcurrent alarms L1']+group['Nr. of overcurrent alarms L2']+group['Nr. of overcurrent alarms L3']
#     group['% of time of feeder overcurrent alarms'] = group['% of time of overcurrent alarms L1']+group['% of time of overcurrent alarms L2']+group['% of time of overcurrent alarms L3']
#     group['Nr. of feeder overcurrent alarms'] = group['Nr. of feeder overcurrent alarms'].sum()
#     group['% of time of feeder overcurrent alarms'] = group['% of time of feeder overcurrent alarms'].sum()

#     group['Nr. of power alarms'] = group['Nr. of power alarms L1']+group['Nr. of power alarms L2']+group['Nr. of power alarms L3']
#     group['% of time of power alarms'] = group['% of time of power alarms L1']+group['% of time of power alarms L2']+group['% of time of power alarms L3']

#     group = group.drop(['Nr. of overcurrent alarms L1','Nr. of overcurrent alarms L2','Nr. of overcurrent alarms L3','% of time of overcurrent alarms L1','% of time of overcurrent alarms L2','% of time of overcurrent alarms L3'],axis=1)
#     group = group.drop(['Nr. of power alarms L1','Nr. of power alarms L2','Nr. of power alarms L3','% of time of power alarms L1','% of time of power alarms L2','% of time of power alarms L3'],axis=1)
#     group = pd.DataFrame(group.iloc[0]).transpose()
#     # columns = group.columns 

#     summary = summary.append(group, ignore_index=True)
#     #print(group.columns)


In [125]:
summary.head()

,Installation,Transformer,Distribution,Total energy consumption (MWh),Nr. of Voltage unbalance alarms,% of time of Voltage unbalance alarms,Nr. of Current unbalance alarms,% of time of Current unbalance alarms,Nr. of Power Fails (outage),Min time of Power Fails (sec),...,Nr. of Voltage swells,Avg time of Voltage swells (msec),Latitude,Longitude,Nr. of feeder overcurrent alarms,Min duration of feeder overcurrent alarms (minutes),Max duration of feeder overcurrent alarms (minutes),Nr. of power alarms,Min duration of power alarms (minutes),Max duration of power alarms (minutes)
0,1-15 ΜΑΡΚΟΠΟΥΛΟΥ,MM-185,NaN,132.92,0.0,0.0,222,1.7,1,41.999,...,0,0.0,37.94104,24.002002,0.0,NaN,NaN,284.0,1.0,112.0
1,Ρ-380 ΣΠΑΤΩΝ,MM-359,NaN,5.84,170.0,1.14,0,0.0,2,264.999,...,0,0.0,37.93041,23.875718,0.0,NaN,NaN,5.0,2.0,84.0
2,Ρ-230 ΝΕΑΣ ΜΑΚΡΗΣ,ΜΓ-019,NaN,8.82,1.0,0.0,0,NaN,1,3018.999,...,0,0.0,38.200179,23.99773,0.0,NaN,NaN,0.0,NaN,NaN
3,Ρ-280 ΛΑΥΡΙΟΥ,ΜΛ-034,NaN,75.96,0.0,0.0,0,0.0,0,0.0,...,0,0.0,37.969795,23.724488,0.0,NaN,NaN,0.0,NaN,NaN
4,1-16 ΜΑΡΚΟΠΟΥΛΟΥ,ΜΜ-014,NaN,16.77,0.0,0.0,0,NaN,0,0.0,...,0,0.0,37.862998,23.85289,0.0,NaN,NaN,0.0,NaN,NaN


In [126]:
summaryCols = summary.drop(['Installation','Transformer','Distribution','Latitude','Longitude'],axis=1).columns
print(summaryCols)

Index(['Total energy consumption (MWh)', 'Nr. of Voltage unbalance alarms',
       '% of time of Voltage unbalance alarms',
       'Nr. of Current unbalance alarms',
       '% of time of Current unbalance alarms', 'Nr. of Power Fails (outage)',
       'Min time of Power Fails (sec)', 'Max time of Power Fails (sec)',
       'Nr. of Voltage dips', 'Avg time of Voltage dips (msec)',
       'Nr. of Voltage swells', 'Avg time of Voltage swells (msec)',
       'Nr. of feeder overcurrent alarms',
       'Min duration of feeder overcurrent alarms (minutes)',
       'Max duration of feeder overcurrent alarms (minutes)',
       'Nr. of power alarms', 'Min duration of power alarms (minutes)',
       'Max duration of power alarms (minutes)'],
      dtype='object')


In [110]:
# summary.rename(columns={'Total energy consumption (KWh)':'Total energy consumption (MWh)'},inplace=True)
# # summaryCols = summary.columns
# summaryCols


In [127]:
summary.to_excel('HEDNO_KPIs_simplified_v3.xlsx', index=False)

In [128]:
summary = summary.fillna(0)

In [129]:
import folium

m = folium.Map(location=[37.98392635351863, 23.727349678741202], tiles="OpenStreetMap", zoom_start=12)



# Add points to the map
for index, row in summary.iterrows():
    
    # tmp = pd.DataFrame(row[['Total energy consumption (KWh)','Nr. of Voltage unbalance alarms','% of time of Voltage unbalance alarms']])#.transpose()
    tmp = pd.DataFrame(row[summaryCols]).transpose()
    # tmp = tmp.reset_index(drop=True)
    # col = tmp.columns[0]
    newcol = row['Installation']+':'+row['Transformer']
    # tmp.rename(columns = {col:newcol},inplace=True)
    
    tmp = tmp.rename(index={tmp.index[0]: newcol})
    

    html = tmp.to_html(
    classes="table table-striped table-hover table-condensed table-responsive"
)
    popup = folium.Popup(html)
    # popup = row['Installation']+'\n'+row['Transformer']+'\n'
    folium.Marker(location=[row['Latitude'], row['Longitude']], popup=popup, icon=folium.Icon()).add_to(m)


m

In [130]:
m.save('HEDNO_KPIs_map_v3.html')
